In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
URL = 'https://egb.com/play/simple_bets'
soup = BeautifulSoup(requests.get(URL).content, 'html.parser')

In [6]:
# for data in soup.html.body:
#     print(data)
#     print(20*'-')  table-bets table-bets--full table-bets--compact
soup.findAll("div", {"class": "table-bets table-bets--full"})

[]

In [7]:
from selenium import webdriver
driver = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver') #executable_path=yourdriver
driver.get(URL)
html = driver.page_source

In [8]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [16]:
table = driver.find_elements_by_class_name('table-bets')
table = table[0].text

In [17]:
soup = BeautifulSoup(table, 'html.parser')

In [19]:
soup.text

"DATE (UTC)\nEVENT\nPLAYER\nODDS\nPLAYER\nODDS\n23.06\n11:00\nLPL\nRoyal Never Give Up\n1.638\nFunPlus Phoenix\n2.106\n23.06\n10:00\nEpicenter\nOG\n1.717\nTNC Predator\n2.101\n23.06\n10:00\nEpicenter\nPSG.LGD\n1.480\nNinjas in Pyjamas\n2.552\n23.06\n10:00\nEpicenter\nVici Gaming\n1.075\nForward Gaming\n4.997\n23.06\n10:00\nEpicenter\nVirtus.pro\n1.147\nAlliance\n4.000\n23.06\n08:00\nLCK\nAfreeca Freecs\n1.329\nGen.G\n2.833\n22.06\n19:00\nCBLOL\nRedemption POA\n1.318\nUppercut esports\n2.877\n22.06\n18:00\nCBLOL\npaiN Gaming\n1.726\nINTZ\n1.993\n22.06\n17:00\nCBLOL\nKaBuM!\n1.075\nCNB\n5.000\n22.06\n16:00\nCBLOL\nFlamengo\n1.075\nTeam oNe\n5.000\n22.06\n10:00\nEpicenter\nTeam Liquid\n1.559\nGambit Esports\n2.365\n22.06\n10:00\nEpicenter\nEvil Geniuses\n1.055\nInFamous\n6.800\n22.06\n10:00\nEpicenter\nEpicenter (picks + bans)\nEpicenter (picks + bans)\n22.06\n10:00\nEpicenter\nRoyal Never Give Up\n1.902\nFnatic\n1.813\n22.06\n10:00\nEpicenter\nTeam Secret\npaiN Gaming\n10.196\n22.06\n09:

In [157]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def remove_header(html, cutoff='RESULTS', separator='\n', padding_token='_PADDING_'):
    tokenized_html = html.split(separator)
    for idx, token in enumerate(tokenized_html):
        if token == cutoff:
            break
    headless_html = padding_token.join(tokenized_html[idx+1:])
    return headless_html
    
def insert_row_breaks(html, separator='_PADDING_', row_break_token='_ROW_BREAK_', padding_token='_PADDING_'):
    tokenized_html = html.split(separator)
    num_tokens = len(tokenized_html)
    insert = False
    processed_html = [tokenized_html[0]]
    for idx in range(1, num_tokens):        
        if insert and not is_number(tokenized_html[idx]):
            insert = False
            processed_html.append(row_break_token)
        elif tokenized_html[idx - 1] == 'Over' and tokenized_html[idx] == 'Under':
            insert = True
            continue
        processed_html.append(tokenized_html[idx])
    processed_html = padding_token.join(processed_html)
    return processed_html

def idx_search(l, s):
    for idx, token in enumerate(l):
        if s in token:
            break
    return idx

def encode_row(row, separator='_PADDING_', stopwords={'_', 'LIVE', '', '-'}):
    # TODO: NA is previous tournament, means no colkumn headers
    
    result = {}
    row = row.split(separator)
    
    time_idx = idx_search(row, ':')
            
    # tournament name
    if len(set(row[:time_idx]) - stopwords) == 1:
        result['tournament_name'] = (set(row[:time_idx]) - stopwords).pop()
    elif len(set(row[:time_idx]) - stopwords) > 1:
        print(set(row[:time_idx]) - stopwords)
        result['tournament_name'] = 'NA'
    else:
        result['tournament_name'] = 'NA'
    
    # time and date
    result['time'] = row[time_idx]
    result['date'] = row[time_idx + 1]
    
    row = row[time_idx + 2:]
    winner_idx = idx_search(row, 'WINNER')
    if 'Total' in row[:winner_idx]:  # bad fit -> correct
        contestants = []
        for winner_idx, token in enumerate(row):
            if is_number(token):
                winner_idx += 1
                break
            contestants.append(token)
        result['contestants'] = contestants
    else:  
        result['contestants'] = row[:winner_idx]
    
    row = row[winner_idx+1:]
    
    # winner odds
    result['winner_odds'] = row[:2]
    
    row = row[2:]
    
    # round handicap
    result['handicap_type'] = row[0]
    result['handicap_odds'] = row[1:5]
    
    row = row[5:]
    
    # map bet
    result['map_type'] = row[0]
    result['map_total'] = row[3]
    result['map_over_odds'] = row[4]
    result['map_under_odds'] = row[5]

    row = row[6:]
    
    print(row)
    
    return result

In [161]:
text = remove_header(soup.text)
text = insert_row_breaks(text)
rows = text.split('_ROW_BREAK_')
for row in rows:
    result = encode_row(row)
    print(result)
#     break

['19', '']
{'tournament_name': 'GG.BET COLOGNE INVITATIONAL', 'time': '05:00', 'date': 'JUN 13', 'contestants': ['Sprout', 'X', 'NoChance'], 'winner_odds': ['1.6', '2.23'], 'handicap_type': '1ST MAP - ROUND HANDICAP (INCL. OVERTIME)', 'handicap_odds': ['-3.5', '2.1', '+3.5', '1.67'], 'map_type': '1ST MAP - TOTAL ROUNDS (INCL. OVERTIME)', 'map_total': '26.5', 'map_over_odds': '1.9', 'map_under_odds': '1.83'}
['19', '']
{'tournament_name': 'GAMEAGENTS LEAGUE SEASON 3', 'time': '05:00', 'date': 'JUN 13', 'contestants': ['Nemiga', 'X', 'Nordavind'], 'winner_odds': ['1.98', '1.76'], 'handicap_type': '1ST MAP - ROUND HANDICAP (INCL. OVERTIME)', 'handicap_odds': ['+3.5', '1.54', '-3.5', '2.35'], 'map_type': '1ST MAP - TOTAL ROUNDS (INCL. OVERTIME)', 'map_total': '26.5', 'map_over_odds': '1.88', 'map_under_odds': '1.85'}
['45', '']
{'tournament_name': 'LOOT.BET SMACK MY BEACH CIS QUALIFIER', 'time': '06:00', 'date': 'JUN 13', 'contestants': ['Syman', 'X', 'NONAME'], 'winner_odds': ['1.1', '6.1

IndexError: list index out of range

In [162]:
row

'_PADDING_LIVE_PADDING_07:00_PADDING_JUN 13_PADDING_AGO_PADDING_X_PADDING_ALTERNATE aTTaX_PADDING_1.59_PADDING_2.25_PADDING_-3.5_PADDING_2.09_PADDING_+3.5_PADDING_1.68_PADDING_Total_PADDING_Over_PADDING_26.5_PADDING_1.9_PADDING_1.83_PADDING_19_PADDING_'

In [160]:
len(rows)

36

In [156]:
text

"GG.BET COLOGNE INVITATIONAL_PADDING_LIVE_PADDING_05:00_PADDING_JUN 13_PADDING_Sprout_PADDING_X_PADDING_NoChance_PADDING_WINNER_PADDING_1.6_PADDING_2.23_PADDING_1ST MAP - ROUND HANDICAP (INCL. OVERTIME)_PADDING_-3.5_PADDING_2.1_PADDING_+3.5_PADDING_1.67_PADDING_1ST MAP - TOTAL ROUNDS (INCL. OVERTIME)_PADDING_Total_PADDING_Over_PADDING_26.5_PADDING_1.9_PADDING_1.83_PADDING_19_PADDING__ROW_BREAK__PADDING_GAMEAGENTS LEAGUE SEASON 3_PADDING_LIVE_PADDING_05:00_PADDING_JUN 13_PADDING_Nemiga_PADDING_X_PADDING_Nordavind_PADDING_WINNER_PADDING_1.98_PADDING_1.76_PADDING_1ST MAP - ROUND HANDICAP (INCL. OVERTIME)_PADDING_+3.5_PADDING_1.54_PADDING_-3.5_PADDING_2.35_PADDING_1ST MAP - TOTAL ROUNDS (INCL. OVERTIME)_PADDING_Total_PADDING_Over_PADDING_26.5_PADDING_1.88_PADDING_1.85_PADDING_19_PADDING__ROW_BREAK__PADDING_LOOT.BET SMACK MY BEACH CIS QUALIFIER_PADDING_LIVE_PADDING_06:00_PADDING_JUN 13_PADDING_Syman_PADDING_X_PADDING_NONAME_PADDING_WINNER_PADDING_1.1_PADDING_6.19_PADDING_MAP HANDICAP_PADDIN

In [65]:
text.split('ROW_BREAK')[0]

'FILTERS_PADDING_MATCH_PADDING_OF THE DAY_PADDING_WINNER_PADDING_Heroic_PADDING_2.15_PADDING_AVANGAR_PADDING_1.65_PADDING_+67_PADDING_UPCOMING GAMES_PADDING_OUTRIGHTS_PADDING_RESULTS_PADDING_GG.BET COLOGNE INVITATIONAL_PADDING_LIVE_PADDING_05:00_PADDING_JUN 13_PADDING_Sprout_PADDING_X_PADDING_NoChance_PADDING_WINNER_PADDING_1.6_PADDING_2.23_PADDING_1ST MAP - ROUND HANDICAP (INCL. OVERTIME)_PADDING_-3.5_PADDING_2.1_PADDING_+3.5_PADDING_1.67_PADDING_1ST MAP - TOTAL ROUNDS (INCL. OVERTIME)_PADDING_Total_PADDING_Over_PADDING_26.5_PADDING_1.9_PADDING_1.83_PADDING_19_PADDING__'

In [19]:
soup.text

"FILTERS\nMATCH\nOF THE DAY\nWINNER\nHeroic\n2.15\nAVANGAR\n1.65\n+67\nUPCOMING GAMES\nOUTRIGHTS\nRESULTS\nGG.BET COLOGNE INVITATIONAL\nLIVE\n05:00\nJUN 13\nSprout\nX\nNoChance\nWINNER\n1.6\n2.23\n1ST MAP - ROUND HANDICAP (INCL. OVERTIME)\n-3.5\n2.1\n+3.5\n1.67\n1ST MAP - TOTAL ROUNDS (INCL. OVERTIME)\nTotal\nOver\nUnder\n26.5\n1.9\n1.83\n19\nGAMEAGENTS LEAGUE SEASON 3\nLIVE\n05:00\nJUN 13\nNemiga\nX\nNordavind\nWINNER\n1.98\n1.76\n1ST MAP - ROUND HANDICAP (INCL. OVERTIME)\n+3.5\n1.54\n-3.5\n2.35\n1ST MAP - TOTAL ROUNDS (INCL. OVERTIME)\nTotal\nOver\nUnder\n26.5\n1.88\n1.85\n19\nLOOT.BET SMACK MY BEACH CIS QUALIFIER\nLIVE\n06:00\nJUN 13\nSyman\nX\nNONAME\nWINNER\n1.1\n6.19\nMAP HANDICAP\n-1.5\n1.42\n+1.5\n2.71\nTOTAL MAPS\nTotal\nOver\nUnder\n2.5\n2.98\n1.36\n45\nGG.BET COLOGNE INVITATIONAL\nLIVE\n06:15\nJUN 13\nLiquid\nX\nSpirit\nWINNER\n1.12\n5.61\n1ST MAP - ROUND HANDICAP (INCL. OVERTIME)\n-7.5\n1.92\n+7.5\n1.81\n1ST MAP - TOTAL ROUNDS (INCL. OVERTIME)\nTotal\nOver\nUnder\n24.5\n1.9